# Генерация текста, count-based подход

скачиваем файлы

In [4]:
!wget https://raw.githubusercontent.com/king-menin/nlp-hse-autumn2018/master/lecture%205.%20language%20models/sonets.txt

--2018-11-15 13:55:04--  https://raw.githubusercontent.com/king-menin/nlp-hse-autumn2018/master/lecture%205.%20language%20models/sonets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96531 (94K) [text/plain]
Saving to: ‘sonets.txt’

sonets.txt          100%[===================>]  94.27K  --.-KB/s    in 0.05s   

2018-11-15 13:55:04 (1.81 MB/s) - ‘sonets.txt’ saved [96531/96531]



In [5]:
!ls

sample_data  sonets.txt


### Предобработка

In [0]:
data_path="sonets.txt"


with open(data_path, "r", encoding="utf-8") as f:
  data = f.read()

#### Удалите все строчки, которые начинаются с Sonnet X. Вставьте в конце каждой части. 

In [0]:
def prc(data, delim="E\n"):
  res = []
  data = data.lower().strip()
  for line in data.split("\n"):
    if not line.startswith("sonnet"):
      if len(line):
        res.append(line + "\n")
      elif len(res):
        if res[-1] != delim:
          res.append(delim)
  res = "".join(res)
  return res, res.split(delim)

In [0]:
text, sonets = prc(data)

In [9]:
print(sonets[0])

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou, contracted to thine own bright eyes,
feed'st thy light'st flame with self-substantial fuel,
making a famine where abundance lies,
thyself thy foe, to thy sweet self too cruel.
thou that art now the world's fresh ornament
and only herald to the gaudy spring,
within thine own bud buriest thy content
and, tender churl, makest waste in niggarding.
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.



In [131]:
print(text[:610])

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou, contracted to thine own bright eyes,
feed'st thy light'st flame with self-substantial fuel,
making a famine where abundance lies,
thyself thy foe, to thy sweet self too cruel.
thou that art now the world's fresh ornament
and only herald to the gaudy spring,
within thine own bud buriest thy content
and, tender churl, makest waste in niggarding.
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.


# Модели биграм в NLTK

In [0]:
import nltk

In [0]:
freq = nltk.FreqDist(text)
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(text))

In [12]:
cfreq['a']

FreqDist({' ': 168,
          "'": 1,
          ',': 4,
          '-': 1,
          'b': 48,
          'c': 177,
          'd': 168,
          'f': 27,
          'g': 88,
          'h': 7,
          'i': 291,
          'j': 2,
          'k': 136,
          'l': 402,
          'm': 135,
          'n': 952,
          'o': 1,
          'p': 66,
          'r': 607,
          's': 387,
          't': 710,
          'u': 127,
          'v': 185,
          'w': 42,
          'y': 214,
          'z': 13})

Получим вероятности биграм

In [13]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(aa) = %1.4f' %cprob['a'].prob('a'))
print('p(ab) = %1.4f' %cprob['a'].prob('b'))
print('p(au) = %1.4f' %cprob['a'].prob('u'))

p(aa) = 0.0000
p(ab) = 0.0097
p(au) = 0.0256


In [14]:
import numpy as np

eps = 0.000001
np.log(cprob['a'].prob('a')+eps) + np.log(cprob['a'].prob('b')+eps) + np.log(cprob['a'].prob('c')+eps)

-21.785947263740503

In [15]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(f) = %1.4f' %unigram_prob('f'))

p(f) = 0.0178


## Задание 1
Напишите функцию для оценки вероятности сонета.

Найдите наиболее вероятный сонет.

In [16]:
cprob["a"].generate()

'n'

In [0]:
seq_len = 100

#### Напишите функцию для оценки вероятности сонета.

In [18]:
# решение 1.1
def sonet_prob(sonet):
    p = unigram_prob(sonet[0])
    for i in range(len(sonet)-1):      
      p *= cprob[sonet[i]].prob(sonet[i+1])
    return p

sonet_prob(sonets[0][:seq_len])

6.677428217395542e-101

In [19]:
print(sonets[0])

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou, contracted to thine own bright eyes,
feed'st thy light'st flame with self-substantial fuel,
making a famine where abundance lies,
thyself thy foe, to thy sweet self too cruel.
thou that art now the world's fresh ornament
and only herald to the gaudy spring,
within thine own bud buriest thy content
and, tender churl, makest waste in niggarding.
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.



Что делать, если вероятность нулевая?

In [20]:
# решение 1.1
def sonet_log_prob(sonet):
    p = unigram_prob(sonet[0])
    for i in range(len(sonet)-1):      
      p += np.log(cprob[sonet[i]].prob(sonet[i+1]) + eps)
    return p

sonet_log_prob(sonets[0])

-1428.8359033355057

## Задание 2
Напишите функцию для генерации нового сонета фиксированной длины (или до токена конца).

In [0]:
# решение 2
def generate_sonet(cprob, first_char, num_chars):
    res = ''
    res += first_char
    for i in range(num_chars):
        char = cprob[first_char].generate()
        res += char
        first_char = char
        if char == "E":
          break
    return res

In [22]:
print(generate_sonet(cprob, 'f', 1000))

fomes, n mmefo kle chamer te shend ternghere, prithe w?
fo ay?
arndisked sthe utonth allas mn thasey d my mit y w, g ild sh's, de pe tho's orecoot ary, tho orrls by murl thathoueere meathiveletot n no w th thy ang ys t h t sptisous,
bas swichimall toubraron d t thimoalot's h cre wo ghethor juns's m ir uthey, sha imu is unosend d bloube d nd.
lowillseneithe owieanofou arep, y s turitthurur vel my trjos thayoug 'e,
all mpld pillinee m'd bundebuno bl thove,
angug le s.
wof vepioug coup chovere wsh whinde ly is buped les ther wees ghar w caus, s t.
' wr
wagoumy chy t sindner dan;
dolican y f hthyst s oour.
ofe ndsth ifrathalin d h thoo wan in armyof,
se-beret s y wis re?
memiser ld murgs henthou re pr'd ty ak nth l
rort.
bl cag ire nd f winoveemeieer wi meis hin derore t tink nosern dsinenthe alst isave fed lt
hace.
tilacaullownesouthikeathesanve tr.
t heangree'silyet lerd t amyergrnd btheatharysthau ng n whincar
iser;
twi litha benge'w, thi bthe fooenoma vetecelloougumy, th t tot terepito

## N-gram word model

In [0]:
words = "\n".join(["S {}".format(s) for s in sonets]).split(" ")
freq = nltk.FreqDist(words)
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(words))
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

#### Функция генерации по словам

In [0]:
def generate_word_sonet(cprob, first_word, num_words=100):
    res = ''
    # res += first_word
    for i in range(num_words):
        word = cprob[first_word].generate()
        if "E" in word:
          break
        if "S" in word:
          continue
        res += " " + word
        first_word = word
    return res.strip()

In [25]:
print(generate_word_sonet(cprob, "S"))

from faring ill.
presume not to have profaned their show; their own thoughts, although their glory die.
the world is most proud as an ever-fixed mark
that looks his presence is, see till action, lust
is perjured, murderous, bloody, full with white;
when lofty trees i am beloved
where i say, within that beauty dyed?
both truth than i was true sight!
or, if it only herald to register,
that may i pray that smells,
if not for eternity,
which prove thee to rehearse?
o, give full of you, but as mine,
robb'd others' voices, that hidden in thy part of this inconstant mind,
since that churl death my sin awards me be


# Генерация текста, нейронные языковые модели

In [26]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


### Подготовка данных

In [0]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [28]:
len(characters)

40

In [0]:
X = []
Y = []
length = len(text)
seq_length = 10
for i in range(0, length-seq_length, 1):
  sequence = text[i:i + seq_length]
  label =text[i + seq_length]
  X.append([char_to_n[char] for char in sequence])
  Y.append(char_to_n[label])

In [37]:
len(X), len(Y)

(94475, 94475)

In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

### Обучение модели

In [0]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def gen_rnn_sonet(string_mapped, model, res_len=500):
  # generating characters
  res = [n_to_char[c] for c in string_mapped]
  for i in range(res_len):
      x = np.reshape(string_mapped,(1,len(string_mapped), 1))
      x = x / float(len(characters))
      pred_index = np.argmax(model.predict(x, verbose=0))
      seq = [n_to_char[value] for value in string_mapped]
      string_mapped.append(pred_index)
      string_mapped = string_mapped[1:len(string_mapped)]
      res.extend(seq)
  return "".join(res)

In [0]:
epochs = 50

In [59]:
print(gen_rnn_sonet(X[np.random.choice(np.arange(len(X)))], model))

ere-througere-througre-throug e-throug t-throug ththroug thehroug the roug the toug the toug the toeg the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toee the toe

In [0]:
for epoch in range(epochs):
  model.fit(X_modified, Y_modified, epochs=1, validation_split=0.05)
  print(gen_rnn_sonet(X[np.random.choice(np.arange(len(X)))], model))

Train on 89751 samples, validate on 4724 samples
Epoch 1/1
89751/89751 [==============================] - 182s 2ms/step - loss: 2.4666 - val_loss: 2.3741
nother thenother theother the ther the wher the woer the worr the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld the wold the word the worl the worldthe world he world te world th world theworld the orld the wrld th

### Несполько часов спустя...

<pre>
the riper should by time decease,
his tender heir might bear his memory:
but thou, contracted to thine own bright eyes,
feed'st thy light's flame with self-substantial fuel,
my beept is she breat oe bath dasehr ill:
tirse do i pine and turfeit day by day,
or gluttoning on all, or all away.
why is my verse so barren of new pride,
so far from variation or quick change?
why with the time do i not glance aside
to new-found methods, and to compounds strange?
why write i stil
</pre>